# Airline Fleet Assignment: An Overview

## Scenario Overview: Optimizing Airline Fleet Assignment

**Decision Variables:**
- $x_{ij}$: Binary variable, 1 if aircraft type $i$ is assigned to flight $j$, and 0 otherwise, where $i$ is an index for aircraft types and $j$ is an index for flights.

**Parameters:**
- $C_{ij}$: Cost of assigning aircraft type $i$ to flight $j$.
- $R_{j}$: Revenue generated from flight $j$.
- $D_{ij}$: Demand for flight $j$ when aircraft type $i$ is used.
- $S_{i}$: Number of aircraft of type $i$ available.
- $M_{ij}$: Maintenance requirement, 1 if aircraft type $i$ needs maintenance after flight $j$, 0 otherwise.
- $T$: Total maintenance slots available.

**Objective Function:**
Maximize total profit, which is the sum of revenues minus costs from assigning aircraft to flights:
$$\text{Maximize } Z = \sum_{i,j} (R_{j} - C_{ij}) \cdot x_{ij}$$

**Constraints:**
1. **Aircraft Assignment Constraint:** Each flight must be assigned exactly one aircraft.
   $$\sum_{i} x_{ij} = 1 \quad \forall j$$
2. **Aircraft Availability Constraint:** The number of flights assigned to each aircraft type cannot exceed its availability.
   $$\sum_{j} x_{ij} \leq S_{i} \quad \forall i$$
3. **Demand Satisfaction Constraint:** The aircraft assigned must meet the demand for each flight.
   $$\sum_{i} D_{ij} \cdot x_{ij} \geq \text{Demand for flight } j \quad \forall j$$
4. **Maintenance Requirement Constraint:** The total maintenance slots used must not exceed the total available.
   $$\sum_{i,j} M_{ij} \cdot x_{ij} \leq T$$


In [2]:
import gurobipy as gp
from gurobipy import GRB

# Create a new model
m = gp.Model("airline_fleet_assignment")

# Data (example values, should be replaced with real data)
aircraft_types = ['A', 'B']
flights = ['F1', 'F2', 'F3']
costs = {'A': {'F1': 100, 'F2': 150, 'F3': 200}, 'B': {'F1': 120, 'F2': 130, 'F3': 180}}
revenues = {'F1': 300, 'F2': 350, 'F3': 400}
demand = {'A': {'F1': 120, 'F2': 150, 'F3': 200}, 'B': {'F1': 100, 'F2': 130, 'F3': 180}}
availability = {'A': 2, 'B': 1}
maintenance = {'A': {'F1': 0, 'F2': 1, 'F3': 0}, 'B': {'F1': 1, 'F2': 0, 'F3': 1}}
total_maintenance_slots = 2

# Variables
x = m.addVars(aircraft_types, flights, vtype=GRB.BINARY, name="assign")

# Objective
m.setObjective(gp.quicksum((revenues[j] - costs[i][j]) * x[i, j] for i in aircraft_types for j in flights), GRB.MAXIMIZE)

# Constraints
# Aircraft Assignment
for j in flights:
    m.addConstr(gp.quicksum(x[i, j] for i in aircraft_types) == 1, name=f"assign_{j}")

# Aircraft Availability
for i in aircraft_types:
    m.addConstr(gp.quicksum(x[i, j] for j in flights) <= availability[i], name=f"avail_{i}")

# Demand Satisfaction
for j in flights:
    m.addConstr(gp.quicksum(demand[i][j] * x[i, j] for i in aircraft_types) >= demand[i][j], name=f"demand_{j}")

# Maintenance Requirement
m.addConstr(gp.quicksum(maintenance[i][j] * x[i, j] for i in aircraft_types for j in flights) <= total_maintenance_slots, name="maintenance")

# Optimize model
m.optimize()

# Display solution
if m.status == GRB.OPTIMAL:
    for v in m.getVars():
        print(f"{v.varName}: {v.x}")

    print(f"Optimal Objective: {m.objVal}")

# Gurobi provides a detailed report after optimization


Restricted license - for non-production use only - expires 2025-11-24
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 9 rows, 6 columns and 21 nonzeros
Model fingerprint: 0x62d709de
Variable types: 0 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [2e+02, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 9 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 620 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.200000000000e+02, best bound 6.200000000000e+02, gap 0.0000%
assign[A,F1]: 1.0
assign[A,F2]: 0.0
assign[A,F3]: 1.0
assign[B,F1]: 0.0
assign[B,F2]